<a href="https://colab.research.google.com/github/Berlingrad/MLinCyberSecurityProject/blob/XuDevelopment/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DNN Back Door Detector

## Preprocessing

###Data Download

In [1]:
!git clone https://github.com/csaw-hackml/CSAW-HackML-2020.git

!gdown https://drive.google.com/uc?id=1P8PTL62x3cfpV9mrC0unqZjRFhlTTOSG
!gdown https://drive.google.com/uc?id=19OKCkY2CjV3ASkOe6nMSYTsOVcxAoCnA
!gdown https://drive.google.com/uc?id=1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV

!mv /content/*.h5 /content/CSAW-HackML-2020/data

##!wget https://raw.githubusercontent.com/bolunwang/backdoor/master/visualizer.py

Cloning into 'CSAW-HackML-2020'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 158 (delta 7), reused 16 (delta 6), pack-reused 141
Receiving objects: 100% (158/158), 79.56 MiB | 37.64 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Downloading...
From: https://drive.google.com/uc?id=1P8PTL62x3cfpV9mrC0unqZjRFhlTTOSG
To: /content/sunglasses_poisoned_data.h5
398MB [00:03, 115MB/s]
Downloading...
From: https://drive.google.com/uc?id=19OKCkY2CjV3ASkOe6nMSYTsOVcxAoCnA
To: /content/clean_validation_data.h5
716MB [00:03, 214MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV
To: /content/clean_test_data.h5
398MB [00:11, 36.0MB/s]


In [2]:
import os
import gdown
import numpy as np
import random
import h5py
import glob2
import tempfile
import copy

import keras
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
!pip install -q tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

if not tf.config.experimental.list_physical_devices('GPU'):
    raise Exception("Please change runtime type to GPU")


     |████████████████████████████████| 174kB 23.7MB/s 


In [3]:
def load_dataset(data_filename, keys=None):
    dataset = {}
    with h5py.File(data_filename, 'r') as hf:
        if keys is None:
            for name in hf:
                dataset[name] = np.array(hf.get(name))
        else:
            for name in keys:
                dataset[name] = np.array(hf.get(name))

    return dataset

def load_models(model_dir):
    models = []
    names = {}

    for i, net in enumerate(glob2.glob("/content/CSAW-HackML-2020/models/*net.h5")):
        print("loading", net)
        weights = glob2.glob(net.split("_")[0]+"*weights.h5")[0]
       
        m = load_model(net)
        m.load_weights(weights)

        models.append(m)
        
        names[i] = os.path.basename(net).split(".")[0]
    return models, names

###Load Data

In [4]:
SG_poisoned_path = "/content/CSAW-HackML-2020/data/sunglasses_poisoned_data.h5"
SG_poisoned = load_dataset(SG_poisoned_path)

clean_validation_path = "/content/CSAW-HackML-2020/data/clean_validation_data.h5"
valid_clean = load_dataset(clean_validation_path)

clean_test_path = "/content/CSAW-HackML-2020/data/clean_test_data.h5"
test_clean = load_dataset(clean_test_path)

SG_poisoned['data'] = SG_poisoned['data'].transpose((0,2,3,1))
valid_clean['data'] = valid_clean['data'].transpose((0,2,3,1))
test_clean['data'] = test_clean['data'].transpose((0,2,3,1))

 
 ##Vectorize label array to fit the model output
outLen = 1283 ##Length of output array
ySG = np.zeros((SG_poisoned['label'].shape[0], outLen))
yValid = np.zeros((valid_clean['label'].shape[0], outLen))
yTest = np.zeros((test_clean['label'].shape[0], outLen))

print(valid_clean['data'].shape)
print(yValid.shape)

for i in range(SG_poisoned['label'].shape[0]):
    ySG[i][SG_poisoned['label'][i]] = 1

for i in range(valid_clean['label'].shape[0]):
    yValid[i][int(valid_clean['label'][i])] = 1

for i in range(test_clean['label'].shape[0]):
    yTest[i][test_clean['label'][i]] = 1



(11547, 55, 47, 3)
(11547, 1283)


####Mixed final test data
Mix the sunglass poinsoned images with clean test images from test set to create a final validation set 

In [5]:
##TODO

###Load Models

In [6]:

model_dir = "/content/CSAW-HackML-2020/models"

models, model_names = load_models(model_dir)

model_names.values()


loading /content/CSAW-HackML-2020/models/anonymous_1_bd_net.h5
loading /content/CSAW-HackML-2020/models/anonymous_2_bd_net.h5
loading /content/CSAW-HackML-2020/models/sunglasses_bd_net.h5
loading /content/CSAW-HackML-2020/models/multi_trigger_multi_target_bd_net.h5


dict_values(['anonymous_1_bd_net', 'anonymous_2_bd_net', 'sunglasses_bd_net', 'multi_trigger_multi_target_bd_net'])

### Model Summary

In [7]:
##TODO
for i, model in enumerate(models):
    print("model ", i, ":")
    model.summary()

     

model  0 :
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 52, 44, 20)   980         input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 26, 22, 20)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 24, 20, 40)   7240        pool_1[0][0]                     
_________________________________________________________________________________

In [8]:
def validate_model(y, y_hat):
    ##compute model accuracy based vectorized output
    n = 0
    for i, v in enumerate(y):
        correct = 1 if np.argmax(v) == np.argmax(y_hat[i]) else 0
        n+= correct 
    return n/(i + 1)
    
    

###Pretrain the models

In [9]:
for m in models:
    pred = m.predict(test_clean['data'])
    print(validate_model(yTest, pred))

for m in models:
    pred = m.predict(SG_poisoned['data'])
    print(validate_model(ySG, pred))



0.6763055339049103
0.6763055339049103
0.42361652377240844
0.4319563522992985
0.0
0.0
0.9994544037412315
0.0


In [10]:
for m in models:
    m.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy']
              )
    m.fit(valid_clean['data'], yValid,
          epochs=2
          )

Epoch 1/2
361/361 [==============================] - 3s 5ms/step - loss: 137.1192 - accuracy: 0.7984
Epoch 2/2
361/361 [==============================] - 2s 5ms/step - loss: 14.6378 - accuracy: 0.8960
Epoch 1/2
361/361 [==============================] - 2s 5ms/step - loss: 144.7029 - accuracy: 0.7912
Epoch 2/2
361/361 [==============================] - 2s 5ms/step - loss: 17.6945 - accuracy: 0.8896
Epoch 1/2
361/361 [==============================] - 2s 4ms/step - loss: 254.6877 - accuracy: 0.6859
Epoch 2/2
361/361 [==============================] - 2s 4ms/step - loss: 5.0150 - accuracy: 0.8590
Epoch 1/2
361/361 [==============================] - 2s 4ms/step - loss: 169.6634 - accuracy: 0.7034
Epoch 2/2
361/361 [==============================] - 2s 4ms/step - loss: 9.9835 - accuracy: 0.8941


In [11]:
preds_preprocessed = [] ##test result of the badnets before repair

for m in models:
    pred = m.predict(test_clean['data'])
    preds_preprocessed.append(pred)
    print(validate_model(yTest, pred))

for m in models:
    pred = m.predict(SG_poisoned['data'])
    print(validate_model(ySG, pred))

0.8521434138737335
0.8582229150428683
0.8194076383476228
0.841153546375682
0.0
0.0
0.3244738893219018
0.0


##Pruning

In [12]:
pruned_models = []
for i, base_model in enumerate(models):
    for _ in range(3):
        model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(base_model)
    
        log_dir = tempfile.mkdtemp()
        callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir)
        ]
        model_for_pruning.compile(
          loss=tf.keras.losses.categorical_crossentropy,
          optimizer='adam',
          metrics=['accuracy']
        )
        model_for_pruning.fit(valid_clean['data'], yValid,
                              callbacks=callbacks,
                              epochs=3
                              )

    pruned_models.append(model_for_pruning)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/3
361/361 [==============================] - 9s 17ms/step - loss: 12.7465 - accuracy: 0.8692
Epoch 2/3
361/361 [==============================] - 6s 17ms/step - loss: 4.8590 - accuracy: 0.9264
Epoch 3/3
361/361 [==============================] - 6s 17ms/step - loss: 2.7142 - accuracy: 0.9478
Epoch 1/3
361/361 [==============================] - 9s 18ms/step - loss: 2.3545 - accuracy: 0.9477
Epoch 2/3
361/361 [==============================] - 6s 17ms/step - loss: 2.5948 - accuracy: 0.9389
Epoch 3/3
361/361 [==============================] - 6s 17ms/step - loss: 1.3291 - accuracy: 0.9575
Epoch 1/3
361/361 [==============================] - 9s 18ms/step - loss: 1.4424 - accuracy: 0.9575
Epoch 2/3
361/361 [==============================] - 6s 17ms/step - loss: 1.7467 - accuracy: 0.9437
Epoch 3/3
361/361 [==============================] - 6s 16ms/step - loss: 1.2682 - accuracy: 0.9549
Epoch 1/3
361/361 [==============================] - 8s 17ms/step - loss: 15.1467 - accuracy: 0.855

In [13]:
'''
pruned_models = []
for i, base_model in enumerate(models):
    model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(base_model)

    loss = tf.keras.losses.categorical_crossentropy
    optimizer = tf.keras.optimizers.Adam()
    log_dir = tempfile.mkdtemp()
    unused_arg = -1
    epochs = 3 
    batches = 1 

    model_for_pruning.optimizer = optimizer
    step_callback = tfmot.sparsity.keras.UpdatePruningStep()
    step_callback.set_model(model_for_pruning)
    log_callback = tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir) # Log sparsity and other metrics in Tensorboard.
    log_callback.set_model(model_for_pruning)

    step_callback.on_train_begin() # run pruning callback
    for _ in range(epochs):
       log_callback.on_epoch_begin(epoch=unused_arg) # run pruning callback
       for _ in range(batches):
           step_callback.on_train_batch_begin(batch=unused_arg) # run pruning callback

           with tf.GradientTape() as tape:
               logits = model_for_pruning(valid_clean['data'], training=True)
               loss_value = loss(yValid, logits)
               grads = tape.gradient(loss_value, model_for_pruning.trainable_variables)
               optimizer.apply_gradients(zip(grads, model_for_pruning.trainable_variables))

       step_callback.on_epoch_end(batch=unused_arg)

    pruned_models.append(model_for_pruning)
'''

"\npruned_models = []\nfor i, base_model in enumerate(models):\n    model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(base_model)\n\n    loss = tf.keras.losses.categorical_crossentropy\n    optimizer = tf.keras.optimizers.Adam()\n    log_dir = tempfile.mkdtemp()\n    unused_arg = -1\n    epochs = 3 \n    batches = 1 \n\n    model_for_pruning.optimizer = optimizer\n    step_callback = tfmot.sparsity.keras.UpdatePruningStep()\n    step_callback.set_model(model_for_pruning)\n    log_callback = tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir) # Log sparsity and other metrics in Tensorboard.\n    log_callback.set_model(model_for_pruning)\n\n    step_callback.on_train_begin() # run pruning callback\n    for _ in range(epochs):\n       log_callback.on_epoch_begin(epoch=unused_arg) # run pruning callback\n       for _ in range(batches):\n           step_callback.on_train_batch_begin(batch=unused_arg) # run pruning callback\n\n           with tf.GradientTape() as tape:\n       

In [14]:
for i, model in enumerate(pruned_models):
    print("model ", i, ":")
    model.summary()

model  0 :
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
prune_low_magnitude_conv_1 (Pru (None, 52, 44, 20)   1942        input[0][0]                      
__________________________________________________________________________________________________
prune_low_magnitude_pool_1 (Pru (None, 26, 22, 20)   1           prune_low_magnitude_conv_1[0][0] 
__________________________________________________________________________________________________
prune_low_magnitude_conv_2 (Pru (None, 24, 20, 40)   14442       prune_low_magnitude_pool_1[0][0] 
_________________________________________________________________________________

##Fine-tuning

In [15]:
repaired_models = []

for i, m in enumerate(pruned_models):

    model = copy.copy(m)

    print("Fine-tune model", model_names[i], ": ")
    model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy']
              )
    model.fit(test_clean['data'], yTest,
          epochs=10
          )
    repaired_models.append(model)

Fine-tune model anonymous_1_bd_net : 
Epoch 1/10
401/401 [==============================] - 8s 13ms/step - loss: 7.1566 - accuracy: 0.8667
Epoch 2/10
401/401 [==============================] - 5s 13ms/step - loss: 6.3895 - accuracy: 0.8760
Epoch 3/10
401/401 [==============================] - 5s 13ms/step - loss: 5.2684 - accuracy: 0.8873
Epoch 4/10
401/401 [==============================] - 5s 13ms/step - loss: 4.5907 - accuracy: 0.9018
Epoch 5/10
401/401 [==============================] - 5s 13ms/step - loss: 4.1400 - accuracy: 0.9043
Epoch 6/10
401/401 [==============================] - 5s 13ms/step - loss: 3.9240 - accuracy: 0.9078
Epoch 7/10
401/401 [==============================] - 5s 13ms/step - loss: 3.6637 - accuracy: 0.9163
Epoch 8/10
401/401 [==============================] - 5s 13ms/step - loss: 3.3261 - accuracy: 0.9178
Epoch 9/10
401/401 [==============================] - 5s 13ms/step - loss: 3.2464 - accuracy: 0.9241
Epoch 10/10
401/401 [==============================] 

##Validation

In [16]:
##Intermediate quick evalution using built-in Keras functions and the poisoned set

for i, m in enumerate(repaired_models):
    print("Repaired", model_names[i], "accuracy on clean validation set: ")
    m.evaluate(test_clean['data'], yTest) 
    print("Repaired", model_names[i], "accuracy on poisoned set: ")
    m.evaluate(SG_poisoned['data'], ySG) 

Repaired anonymous_1_bd_net accuracy on clean validation set: 
401/401 [==============================] - 2s 4ms/step - loss: 2.5543 - accuracy: 0.9350
Repaired anonymous_1_bd_net accuracy on poisoned set: 
401/401 [==============================] - 2s 4ms/step - loss: 1740.5236 - accuracy: 0.0000e+00
Repaired anonymous_2_bd_net accuracy on clean validation set: 
401/401 [==============================] - 2s 4ms/step - loss: 2.5896 - accuracy: 0.9267
Repaired anonymous_2_bd_net accuracy on poisoned set: 
401/401 [==============================] - 2s 4ms/step - loss: 1345.1877 - accuracy: 0.0000e+00
Repaired sunglasses_bd_net accuracy on clean validation set: 
401/401 [==============================] - 2s 4ms/step - loss: 0.7017 - accuracy: 0.8891
Repaired sunglasses_bd_net accuracy on poisoned set: 
401/401 [==============================] - 2s 4ms/step - loss: 19.4443 - accuracy: 0.0087
Repaired multi_trigger_multi_target_bd_net accuracy on clean validation set: 
401/401 [============

##Final output
Compare preprocessed results from the bad net and results from repaired nets to output results. The differnt predictions between bad nets and repaired nets are classifed as N+1 class.

In [17]:
def poisoned_detection(afterPreds, prePreds):
    out = []
    for i in range(len(afterPreds)):
        newDimension = list(afterPreds[i].shape)
        newDimension[1] += 1
        newArray = np.zeros(newDimension)
        for j in range(afterPreds[i].shape[0]):
            a = np.argmax(afterPreds[i][j]) 
            b = np.argmax(prePreds[i][j])
            
            if a == b:
                newArray[j][a] = 1
            else:
                newArray[j][-1] = 1
        out.append(newArray)

    return out

In [18]:
preds_repaired = [] ##test result of the badnets after repair

for m in repaired_models:
    pred = m.predict(test_clean['data'])
    preds_repaired.append(pred)
    print(validate_model(yTest, pred))

final_result = poisoned_detection(preds_repaired, preds_preprocessed)

0.9349961028838659
0.9266562743569758
0.8890880748246298
0.9151987529228371
